<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/BERT_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Based Sentiment Analysis <br>
*original is at: https://www.kaggle.com/pastrop/toxic-data-comp-data*

In [1]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 2.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=49e8af10d285b2d5066e336067e0ad752c95b7da643479447ab7b3cb0f3533af
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=34e727439e8e064a1cc083359e88d7c18b73b9a943d58070bd98483098820d12
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=d650598f206d2e9079ea0bebd1058d5b381305f12da69f94422a474cae679fa7
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [0]:
#import keras-bert
#from keras_bert import Tokenizer
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/2",trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

In [0]:
import bert
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
# Data preproessing (depends on Kaggle Data)
SEQUENCE_LENGTH = 128
DATA_PATH =  "../input/jigsaw-multilingual-toxic-comment-classification"

In [0]:
def process_sentence(sentence, max_seq_length=SEQUENCE_LENGTH, tokenizer=tokenizer):
    """Helper function to prepare data for BERT. Converts sentence input examples
    into the form ['input_word_ids', 'input_mask', 'segment_ids']."""
    # Tokenize, and truncate to max_seq_length if necessary.
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    # Convert the tokens in the sentence to word IDs.
    input_ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"])

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    pad_length = max_seq_length - len(input_ids)
    input_ids.extend([0] * pad_length)
    input_mask.extend([0] * pad_length)

    # We only have one input segment.
    segment_ids = [0] * max_seq_length

    return (input_ids, input_mask, segment_ids)

def preprocess_and_save_dataset(unprocessed_filename, text_label='comment_text',
                                seq_length=SEQUENCE_LENGTH, verbose=True):
    """Preprocess a CSV to the expected TF Dataset form for multilingual BERT,
    and save the result."""
    dataframe = pandas.read_csv(os.path.join(DATA_PATH, unprocessed_filename),
                                index_col='id')
    processed_filename = (unprocessed_filename.rstrip('.csv') +
                          "-processed-seqlen{}.csv".format(SEQUENCE_LENGTH))

    pos = 0
    start = time.time()

    while pos < len(dataframe):
        processed_df = dataframe[pos:pos + 10000].copy()

        processed_df['input_word_ids'], processed_df['input_mask'], processed_df['all_segment_id'] = (
            zip(*processed_df[text_label].apply(process_sentence)))

        if pos == 0:
            processed_df.to_csv(processed_filename, index_label='id', mode='w')
        else:
            processed_df.to_csv(processed_filename, index_label='id', mode='a',
                                header=False)

        if verbose:
            print('Processed {} examples in {}'.format(
                pos + 10000, time.time() - start))
        pos += 10000
    return
  
# Process the training dataset.
preprocess_and_save_dataset(wiki_toxic_comment_data)